In [1]:
import tensorflow as tf
import numpy as np

In [2]:
def make_vocab(vocab, pad, start, end, unk):
    
    t2i = {'<PAD>': pad, '<BOS>': start, '<EOS>': end, '<UNK>': unk}
    i2t = {pad: '<PAD>', start: '<BOS>', end: '<EOS>', unk: '<UNK>'}
    
    for word, idx in vocab.items():
        t2i[word] = idx + 3
        i2t[idx + 3] = word
        
    return t2i, i2t

In [3]:
def load_data(file_path):
    '''
    Load numpy data
    
    Args:
        inputs: json data type, key => title, value => poem
    
    
    Return:
        inputs
        vocab
    '''
    
    pad_token = 0 # <PAD>
    start_token = 1 # <BOS>
    end_token = 2 # <EOS>
    unk_token = 3 #<UNK>
    
    
    data = json.load(open(file_path, 'r'))
    data_list = []
    all_sentences = []
    
    for poem in data.values():
        data_list.append(poem)
        all_sentences.extend(poem)
    
    source = []
    target = []
    
    for item in data_list:
        source.extend(item[:-1])
        target.extend(item[1:])
        
    max_len = int(round(np.array([len(x.split(' ')) for x in all_sentences]).mean()))
    
    tokenizer = tf.keras.preprocessing.text.Tokenizer()
    tokenizer.fit_on_texts(all_sentences)
    source = tokenizer.texts_to_sequences(source)
    target = tokenizer.texts_to_sequences(target)
    
    assert len(source) == len(target)
    
    for i in range(len(source)):
        source[i] = np.hstack(([start_token], np.add(source[i][:max_len], 3), [end_token]))
        target[i] = np.hstack(([start_token], np.add(target[i][:max_len], 3), [end_token]))    
            
    inputs = tf.keras.preprocessing.sequence.pad_sequences(source, maxlen=max_len + 2, padding = 'post')
    labels = tf.keras.preprocessing.sequence.pad_sequences(target, maxlen=max_len + 2, padding = 'post')
    
    t2i, i2t = make_vocab(tokenizer.word_index, pad_token, start_token, end_token, unk_token)
    
    
    return inputs, labels, t2i, i2t

In [4]:
def prepare_data(inputs, labels):
    
    encoder_inputs = inputs[:,1:]
    decoder_inputs = labels[:, :-1]
    decoder_outputs = labels[:, 1:]
    
    return encoder_inputs, decoder_inputs, decoder_outputs

In [5]:
def losd_numpy_array(path):
    return np.load(open(path, 'rb'))

In [6]:
def positional_encoding(sentence_len, model_dim, dtype = tf.float32):
    '''
    Positional Encoding
    paper: https://arxiv.org/abs/1706.03762
    
    arg
        sentence_len: integer.
        model_dim: integer. dimension used by model
        dtype: tensorflow data type Default is 'tf.float32'
    
    return
        positional_2d: shape is [sentence_len, model_dim]
    '''
    
    positional_1d = np.array([pos/10000**(2*i/model_dim) for pos in range(sentence_len) for i in range(model_dim)])
    # for even number
    positional_1d[::2] = np.sin(positional_1d[::2])
    positional_1d[1::2] = np.cos(positional_1d[1::2])
    
    positional_2d = positional_1d.reshape([sentence_len, model_dim])
    
    
    
    return positional_2d 

In [7]:
def multi_head_attention(queries, 
                            keys,
                            values,
                            k_dim,
                            v_dim,
                            num_heads = 8,
                            num_units = None, 
                            drop_out = 0, 
                            is_train = True, 
                            scope = "multi_head_attention", 
                            reuse = None, 
                            masked = False):
    
    '''
    Multi head Attention
    
    Input
    queries => [BS x sentence len x d_model]
    keys => [BS x sentence len x d_model]
    value => [BS x sentence len x d_moel]
    num_heads => number of head
    k_dim => dimension of k
    v_dim => dimension of v
    '''
    
    assert k_dim % num_heads == 0, 'Dimension can\'t devided by number of head' 
    assert v_dim % num_heads == 0, 'Dimension can\'t devided by number of head' 
    
    
    with tf.variable_scope(scope):
        
        d_model = queries.shape.as_list()[-1]
        batch_size = queries.shape.as_list()[0]
        sentence_len = queries.shape.as_list()[1]
        
        '''
        Q => [ BS x sentence len x k_dim ]
        K => [ BS x sentence len x k_dim ]
        V => [ BS x sentence len x v_dim ]
        '''
        Q = tf.keras.layers.Dense(units = k_dim)(queries)
        K = tf.keras.layers.Dense(units = k_dim)(keys)
        V = tf.keras.layers.Dense(units = v_dim)(values)
        
        '''
        Masking 추가 필요
        '''
        
        k_dim_i = k_dim // num_heads 
        v_dim_i = v_dim // num_heads
        
        Q_i = tf.stack(tf.split(value = Q, num_or_size_splits = num_heads, axis = -1)) # [num_head x BS x sentence len x (k_dim/num_head)]
        K_i = tf.stack(tf.split(value = K, num_or_size_splits = num_heads, axis = -1)) # [num_head x BS x sentence len x (k_dim/num_head)]
        V_i = tf.stack(tf.split(value = V, num_or_size_splits = num_heads, axis = -1)) # [num_head x BS x sentence len x (v_dim/num_head)]
        
        Q_K = tf.matmul(Q_i, tf.transpose(K_i, [0,1,3,2])) / (K_i.shape.as_list()[-1] ** 0.5) # [num_head x BS x sentence_len x sentence_len]

        
        #if masked:
        #    output_like_softmax = tf.ones_like(Q_K[0, 0, :, :])
        #    lower_triangle = tf.linalg.LinearOperatorLowerTriangular(output_like_softmax).to_dense()
        #    masks = tf.tile(tf.expand_dims(tf.expand_dims(lower_triangle, 0), 0), [Q_K.shape.as_list()[0], Q_K.shape.as_list()[1], 1, 1])
        #    pad = tf.ones_like(masks) * -(2**32 +1)
        #    Q_K = tf.where(tf.equal(masks, 0), pad, Q_K)
        
        if masked:
            masks = tf.linalg.LinearOperatorLowerTriangular(Q_K).to_dense()
            pad = tf.ones_like(masks) * -(2**32 +1)
            Q_K = tf.where(tf.equal(masks, 0), pad, Q_K)
        
        Q_K_softmax = tf.nn.softmax(Q_K) # [num_head x BS x sentence_len x sentence_len]
        outputs = tf.matmul(Q_K_softmax, V_i) # [num_head x BS x sentence len x (v_dim/num_head)]
            
        outputs = tf.transpose(outputs,[1,2,0,3])
        shape_outputs = outputs.get_shape().as_list() # [?, 7, 8, 64]
        outputs = tf.reshape(outputs, shape = [-1, shape_outputs[1], shape_outputs[2] * shape_outputs[3]])
        outputs = tf.keras.layers.Dense(units = d_model)(outputs)
        
        
        
        return outputs

In [8]:
def add_and_normalize(x, f_x, scpoe = "add_and_normalize"):
    return tf.contrib.layers.layer_norm(tf.add(x,f_x))

In [9]:
def feed_forward_layer(inputs, w1_dim, w2_dim, scope = "feed_forward_layer"):

    with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
        x = tf.keras.layers.Dense(units=w1_dim, activation=tf.nn.relu)(inputs)
        out = tf.keras.layers.Dense(units=w2_dim)(x)
        
    return out

In [10]:
def conv_1d_layer(inputs, num_filter1, num_filter2, scope = "conv_1d_layer"):
    
    with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
        x = tf.keras.layers.Conv1D(filters = num_filter1, kernel_size = 1, activation = tf.nn.relu)(inputs)
        out = tf.keras.layers.Conv1D(filters = num_filter2, kernel_size = 2)(inputs)
    
    return out

In [11]:
def encoder(encoder_inputs, num_layers, k_dim, v_dim, num_heads, w1_dim = None, w2_dim = None,
            dropout_rate = 0.2, use_conv = False):
    
    ## is_train 추가필요, 우선 어디 사용되는지 확인 필요
    
    
    '''
    Args:
        encoder_inputs: embeded and encoded inputs, shape => [BS x sentence_len x model_dim]
        num_layers: integer, number of layers
        k_dim: integer, dimension of keys and queries
        v_dim: integer, dimension of values
        num_heads: integer, number of head
        w1_dim: integer, number of units in first feed forward layer
        w_2dim: integer, number of units in second feed forward layer. It must be same with dimension of model
        dropout_rate: float, it must be in [0, 1]. Dropout rate
        embedding_matrix: matrix with shape [vocab_size x model_dim], It's used when model use pre-train embedding vector
        vocab_size: integer. If it doesn't use pre-train embedding. then have to put vocab_size
        embedding_dimension: integer. If it doesn't use pre-train embedding. then have to put embedding_dimension
        use_conv: Bool, If True, use conv-1d, instead of feed forward layer
        
    Returns:
        encoder_outputs: matrix with shape [BS x sentence_len x model_dim]
    '''
    
    
    with tf.variable_scope("encoder", reuse = False):
        
        model_dim = encoder_inputs.shape.as_list()[2]
        
        encoder_inputs = tf.keras.layers.Dropout(rate = dropout_rate)(encoder_inputs)
        
        if w1_dim is None:
            w1_dim = model_dim * 4
        if w2_dim is None:
            w2_dim = model_dim
        
        for i in range(num_layers):
            with tf.variable_scope("encoder_{}-layer".format(i)):
                encoder_inputs_attention = multi_head_attention(queries = encoder_inputs,
                                                                 keys = encoder_inputs,
                                                                 values =  encoder_inputs,
                                                                 k_dim = k_dim,
                                                                 v_dim = v_dim,
                                                                 num_heads = num_heads,
                                                                 drop_out = dropout_rate,
                                                                 masked = True)
                encoder_inputs = add_and_normalize(x = encoder_inputs, f_x = encoder_inputs_attention)
                
                if use_conv:
                    encoder_inputs_forward = conv_1d_layer(inputs = encoder_inputs, num_filter1 = w1_dim, num_filter2 = w2_dim)
                else:
                    encoder_inputs_forward = feed_forward_layer(inputs = encoder_inputs, w1_dim = w1_dim, w2_dim = w2_dim)
                encoder_inputs = add_and_normalize(x = encoder_inputs, f_x = encoder_inputs_forward)
            
                    
        encoder_ouputs = encoder_inputs        
        
        return encoder_ouputs

In [12]:
def decoder(encoder_outputs, decoder_inputs, num_layers, k_dim, v_dim, num_heads, w1_dim = None, w2_dim = None,
            dropout_rate = 0.2, use_conv = False):
    
    '''
    Args:
        encoder_inputs: embeded and encoded inputs, shape => [BS x sentence_len x model_dim]
        num_layers: integer, number of layers
        k_dim: integer, dimension of keys and queries
        v_dim: integer, dimension of values
        num_heads: integer, number of head
        w1_dim: integer, number of units in first feed forward layer
        w_2dim: integer, number of units in second feed forward layer. It must be same with dimension of model
        dropout_rate: float, it must be in [0, 1]. Dropout rate
        embedding_matrix: matrix with shape [vocab_size x model_dim], It's used when model use pre-train embedding vector
        vocab_size: integer. If it doesn't use pre-train embedding. then have to put vocab_size
        embedding_dimension: integer. If it doesn't use pre-train embedding. then have to put embedding_dimension
        use_conv: Bool, If True, use conv-1d, instead of feed forward layer
        
    Returns:
        encoder_outputs: matrix with shape [BS x sentence_len x model_dim]
    '''
    
    with tf.variable_scope("decoder", reuse = False):
        
        model_dim = decoder_inputs.shape.as_list()[2]
        
        decoder_inputs = tf.keras.layers.Dropout(rate = dropout_rate)(decoder_inputs)
        
        if w1_dim is None:
            w1_dim = model_dim * 2
        if w2_dim is None:
            w2_dim = model_dim
        
        
        for i in range(num_layers):
            with tf.variable_scope("decoder_{}-layer".format(i)):
                decoder_self_attention = multi_head_attention(queries = decoder_inputs,
                                                                 keys = decoder_inputs,
                                                                 values =  decoder_inputs,
                                                                 k_dim = k_dim,
                                                                 v_dim = v_dim,
                                                                 num_heads = num_heads,
                                                                 drop_out = dropout_rate,
                                                                 masked = True)
                
                decoder_inputs = add_and_normalize(x = decoder_inputs, f_x = decoder_self_attention)
                
                decoder_attention = multi_head_attention(queries = encoder_outputs,
                                                            keys = encoder_outputs,
                                                            values = decoder_inputs,
                                                            k_dim = k_dim,
                                                            v_dim = v_dim,
                                                            num_heads = num_heads,
                                                            drop_out = dropout_rate,
                                                            masked = True)
                
                decoder_inputs = add_and_normalize(x = decoder_inputs, f_x = decoder_attention)
                
                if use_conv:
                    decoder_inputs_forward = conv_1d_layer(inputs = decoder_inputs, num_filter1 = w1_dim, num_filter2 = w2_dim)
                else:
                    decoder_inputs_forward = feed_forward_layer(inputs = decoder_inputs, w1_dim = w1_dim, w2_dim = w2_dim)
                
                decoder_inputs = add_and_normalize(x = decoder_inputs, f_x = decoder_inputs_forward)
    
        decoder_outputs = decoder_inputs
        
        return decoder_outputs

In [61]:
def model(features, mode, params):
    
    embedding_matrix = params['embedding_matrix']
    VOCAB_SIZE = params['vocab_size']
    EMBEDDING_DIMENSION = params['embedding_dimension']
    NUM_LAYERS = params['num_layers']
    K_DIM = params['k_dim']
    V_DIM = params['v_dim']
    NUM_HEADS = params['num_heads']
    W1_DIM = params['w1_dim']
    W2_DIM = params['w2_dim']
    DROPOUT_RATE = params['dropout_rate']
    USE_CONV = params['use_conv']
    SENTENCE_LEN = params['sentence_len']
    LEARNING_RATE = params['learning_rate']
    
    
    encoder_inputs = features['encoder_inputs']
    decoder_inputs = features['decoder_inputs']
    labels = features['decoder_labels']
    
    batch_size = tf.shape(encoder_inputs)[0]
    
    TRAIN = mode == tf.estimator.ModeKeys.TRAIN
    EVAL = mode == tf.estimator.ModeKeys.EVAL
    PREDICT = mode == tf.estimator.ModeKeys.PREDICT
    
    
    if embedding_matrix is None:
        embedding_matrix = tf.get_variable(name = 'embedding_matrix',
                                           dtype=tf.float32,
                                           shape=[VOCAB_SIZE, EMBEDDING_DIMENSION],
                                           initializer = tf.contrib.layers.xavier_initializer())
    # if zero_pad:
    # 
    
    encoder_inputs = tf.nn.embedding_lookup(ids = encoder_inputs, params = embedding_matrix)
    decoder_inputs = tf.nn.embedding_lookup(ids = decoder_inputs, params = embedding_matrix)
    
    encoder_inputs = tf.add(encoder_inputs,
                            positional_encoding(sentence_len = SENTENCE_LEN, model_dim = EMBEDDING_DIMENSION, dtype = tf.float32))
    decoder_inputs = tf.add(decoder_inputs,
                            positional_encoding(sentence_len = SENTENCE_LEN, model_dim = EMBEDDING_DIMENSION, dtype = tf.float32))
    
    #embedding함수 지우고 dropout 추가필요
    
    encoder_outputs = encoder(encoder_inputs = encoder_inputs,
                              num_layers = NUM_LAYERS,
                              k_dim = K_DIM,
                              v_dim = V_DIM,
                              num_heads = NUM_HEADS,
                              w1_dim = W1_DIM,
                              w2_dim = W2_DIM,
                              dropout_rate = DROPOUT_RATE,
                              use_conv = USE_CONV)
    
    decoder_outputs = decoder(encoder_outputs = encoder_outputs,
                              decoder_inputs = decoder_inputs,
                              num_layers = NUM_LAYERS,
                              k_dim = K_DIM,
                              v_dim = V_DIM,
                              num_heads = NUM_HEADS,
                              w1_dim = W1_DIM,
                              w2_dim = W2_DIM,
                              dropout_rate = DROPOUT_RATE,
                              use_conv = USE_CONV)
    
    embedding_matrix_tile = tf.tile(tf.expand_dims(embedding_matrix, 0), [batch_size, 1, 1])
    
    linear_outputs = tf.matmul(decoder_outputs, embedding_matrix_tile, transpose_b = True) # [BS x sen_len x vocab_size]
    
    prob_outputs = tf.nn.softmax(linear_outputs)
    token_outputs = tf.argmax(prob_outputs, axis = -1)
    if PREDICT:
        
        return tf.estimator.EstimatorSpec(mode = mode, predictions = token_outputs)
        
    mask_zero = 1 - tf.cast(tf.equal(labels, 0),dtype=tf.float32)
    mask_end = 1 - tf.cast(tf.equal(labels, 2), dtype=tf.float32)
    labels_one_hot = tf.one_hot(indices = labels, depth = VOCAB_SIZE, dtype = tf.float32) # [BS, senxlen, vocab_size]
    loss = tf.nn.softmax_cross_entropy_with_logits(labels = labels_one_hot, logits = linear_outputs)
    loss = loss * mask_zero
    loss = loss * mask_end
    loss = tf.reduce_mean(loss)
    
    
    
    if EVAL:
        return tf.estimator.EstimatorSpec(mode = mode, loss = loss)
    
    else:
        
        optimizer = tf.train.AdamOptimizer(learning_rate = LEARNING_RATE)
        train_optimizer = optimizer.minimize(loss, global_step  = tf.train.get_global_step())
        
        return tf.estimator.EstimatorSpec(mode = mode, loss = loss, train_op = train_optimizer)
    

    # label smoothing 기법 추가필요

In [62]:
def make_pred_decoder_inputs(input_token, output_token, idx):
    
    '''
    idx: 1->6
    '''
    
    
    # input_token, output_token, idx
    # [1,0,0,0,0,0,0], X, [1 , 32, X, X, X, X]
    # [1,32,0,0,0,0,0], [1, 32, 77, X, X, X, X], 1
    # [1,32,77,0,0,0,0], []
    # [1,32,77,5,0,0,0]
    # [1,32,77,5,39,0,0]
    # [1,32,77,5,39,8,0]
    
    batch_size = input_token.shape.as_list()[0]
    max_len = input_token.shape.as_list()[1]
    
    left = tf.slice(input_token, [0,0], [batch_size, idx])
    right = tf.slice(output_token [0, idx], [batch_size, 1])
    zero = tf.zeros_like(input_token)
    zero_slice = tf.slice(zero, [0, idx + 1], [batch_size, max_len - (idx + 1)])
    
    new_input = tf.concat((left,right,zero_slice), axis=1)
    
    return new_input

In [63]:
import json
import time
import pandas as pd

In [64]:
PATH = './poem_data.json'

inputs, labels, t2i, i2t = load_data(PATH)
encoder_inputs, decoder_inputs, decoder_labels = prepare_data(inputs, labels)

In [65]:
def mapping_fn(X, Y, Z):
    features = {'encoder_inputs': X, 'decoder_inputs': Y, 'decoder_labels': Z}
    return features

def train_data():
    
    dataset = tf.data.Dataset.from_tensor_slices((encoder_inputs, decoder_inputs, decoder_labels))
    dataset = dataset.shuffle(buffer_size=len(encoder_inputs))
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(mapping_fn)
    dataset = dataset.repeat(count=NUM_EPOCH)
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

In [66]:
MODEL_DIR = './check_point'
params = {'embedding_matrix': None,
          'vocab_size': len(i2t),
          'embedding_dimension': 512,
          'num_layers': 6,
          'k_dim': 512,
          'v_dim': 512,
          'num_heads': 8,
          'w1_dim': None,
          'w2_dim': None,
          'dropout_rate': 0.2,
          'use_conv': False,
          'sentence_len': encoder_inputs.shape[1],
          'learning_rate': 0.001}

est = tf.estimator.Estimator(model, model_dir=MODEL_DIR, params=params)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './check_point', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000189A16D4EF0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [67]:
BATCH_SIZE = 16
NUM_EPOCH = 2

est.train(train_data)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./check_point\model.ckpt.
INFO:tensorflow:loss = 5.336877, step = 1
INFO:tensorflow:Saving checkpoints for 96 into ./check_point\model.ckpt.
INFO:tensorflow:Loss for final step: 6.020566.


In [68]:
pred_encoder_inputs, pred_decoder_inputs, pred_decoder_outputs = np.array([encoder_inputs[38]]), np.array([decoder_inputs[38]]), np.array([decoder_inputs[38]])

In [69]:
def pred_input_fn():
    
    dataset = tf.data.Dataset.from_tensor_slices((pred_encoder_inputs, pred_decoder_inputs, pred_decoder_outputs))
    dataset = dataset.batch(1)
    dataset = dataset.map(mapping_fn)
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

In [70]:
predict = [p for p in est.predict(input_fn = pred_input_fn)]

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./check_point\model.ckpt-96
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [71]:
predict

[array([5, 5, 5, 5, 5, 5, 5], dtype=int64)]

In [59]:
764//16

47